In [ ]:
# Standard Library Imports
import time
import colorsys
from itertools import combinations
from collections import defaultdict

# Scientific Computing
import numpy as np
import sympy as sp
from sympy import symbols, diff
from scipy.spatial import ConvexHull

# Plotting Libraries
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import plotly.graph_objects as go

# PyCalphad (Thermodynamics Calculations & Plotting)
from pycalphad import Database, calculate, equilibrium, variables as v, Model, ternplot
from pycalphad.plot.utils import phase_legend

# Computational Geometry
from shapely.geometry import Polygon, MultiPolygon
from shapely.ops import unary_union

# Color Processing
from skimage.color import deltaE_ciede2000, rgb2lab

In [ ]:
# Load database and choose the phases that will be plotted
db = Database(r'../TDDatabaseFiles_temp/alfe.tdb')

phases = list(db.phases.keys())
constituents = list(db.elements)
legend_handles, color_dict = phase_legend(phases)

print(phases)
print(constituents)

In [ ]:
# Remove the sublattice phases
for phase in ['AL13FE4', 'AL5FE2', 'B2_BCC', 'AL2FE']:
    if phase in phases:
        phases.remove(phase)

In [ ]:
def fit_poly_points(x_data, y_data):
    # Construct the design matrix
    A = np.column_stack([
        np.ones_like(x_data), 
        x_data,               
        x_data**2       
    ])
    
    # Solve the least squares problem to get the coefficients
    coeffs, residuals, rank, s = np.linalg.lstsq(A, y_data, rcond=None)
    
    # Create sympy symbols for x and y
    x= sp.symbols('x')
    
    # Define the list of polynomial terms in the same order as in A:
    terms = [
        1,         
        x, 
        x**2
    ]
    
    # Build the polynomial expression by summing coeff * term for each term.
    expr = sum(sp.Float(coeff) * term for coeff, term in zip(coeffs, terms))
    
    return sp.simplify(expr), residuals

In [ ]:
fig = plt.figure(figsize=(6,6))
ax = fig.gca()

# Loop over phases
for phase in phases:
    result = calculate(db, constituents, phase, T=298.15, P=101325, output='GM')
    try:
        ax.scatter(result.X.sel(component='FE'), result.GM, marker='.', s=20, color=color_dict[phase], label=phase)
    except KeyError:
        pass

# Format the plot
ax.set_xlabel('X(FE)')
ax.set_ylabel('GM')
ax.set_xlim((0, 1))
ax.legend(loc='center left', bbox_to_anchor=(1, 0.6))
ax.set_title('Al-Fe Phases')
plt.show()

In [ ]:
# Loop over phases
phase_poly_dict = dict()
for phase in phases:
    result = calculate(db, constituents, phase, T=298.15, P=101325, output='GM')
    x_data = result.X.sel(component='FE').values.ravel()
    y_data = result.GM.values.ravel()

    poly = fit_poly_points(x_data, y_data)[0]
    # Round the polynomial
    # poly = poly.replace(lambda term: term.is_Number, lambda term: int(round(term, 0)))
    phase_poly_dict[phase] = poly


fig = plt.figure(figsize=(6,6))
ax = fig.gca()

# Loop over phases
x_vals = np.linspace(0, 1, 100)
for phase in phases:
    g_data = sp.lambdify(symbols('x'), phase_poly_dict[phase], 'numpy')(x_vals)
    try:
        ax.plot(x_vals, g_data, color=color_dict[phase], label=phase, linewidth=4)
    except KeyError:
        pass

# Format the plot
ax.set_xlabel('X(FE)')
ax.set_ylabel('GM')
ax.set_xlim((0, 1))
ax.legend(loc='center left', bbox_to_anchor=(1, 0.6))
ax.set_title('Al-Fe Phases')
plt.show()

In [ ]:
def quad_discriminant(expr, var):
    # Extract coefficients of x^2, x, and constant term
    collection = sp.collect(expr, var)
    a = collection.coeff(var, 2)  # Coefficient of x^2
    b = collection.coeff(var, 1)  # Coefficient of x
    c = collection.coeff(var, 0)  # Constant term

    # Compute the discriminant manually
    d = b**2 - 4*a*c

    return d

def projection_function(f1, f2):
    '''
    Computes the projection of f2 onto f1.
    
    Parameters:
        f1 : sympy expression

        f2 : sympy expression
    
    Returns:
        proj : sympy expression

        variables : tuple

        pvariables : tuple
    '''
    free_symbols_f1 = f1.free_symbols
    free_symbols_f2 = f2.free_symbols

    variables = tuple(free_symbols_f1.intersection(free_symbols_f2))
    pvariables = tuple(symbols(f'{symbol}_p') for symbol in variables)

    sum_terms = [(variable - pvariables[i])*diff(f1, variable).subs(variable, pvariables[i]) for i, variable in enumerate(variables)]
    proj = f2 - f1.subs(dict(zip(variables, pvariables))) - sum(sum_terms)

    return proj.expand(), variables, pvariables

def recursive_discriminant(expr, vars):
    '''
    Takes the discriminant over all variables in a given expression.

    Parameters:
        expr (sympy expression): The expression to take the discriminant of.

    Returns:
        discriminant (sympy expression): The discriminant of the expression.
    '''
    vars_list = list(vars)
    def recursive_discriminant_helper(expr, vars):
        # if there are no variables, return the expression
        if vars == []:
            return expr
        
        for var in vars_list:
            if expr.has(var):
                vars_list.remove(var)
                return quad_discriminant(recursive_discriminant(expr, vars_list), var)

            
    return recursive_discriminant_helper(expr, vars_list)

In [ ]:
# Loop over phases
x_vals = np.linspace(0, 1, 100)
pairs = list(combinations(phase_poly_dict.keys(), 2))
# pairs = [('FCC_A1', 'AL5FE4')]
display(pairs)

In [ ]:
phase_values_dict = defaultdict(list)
for pair in pairs:
    phase1 = pair[0]
    phase2 = pair[1]

    proj, variables, pvariables = projection_function(phase_poly_dict[phase1], phase_poly_dict[phase2])
    discriminant = recursive_discriminant(proj, variables)
    sols = sp.solve(discriminant, pvariables)

    phase_values_dict[phase1].extend(sols)

    proj, variables, pvariables = projection_function(phase_poly_dict[phase2], phase_poly_dict[phase1])
    discriminant = recursive_discriminant(proj, variables)
    sols = sp.solve(discriminant, pvariables)
    phase_values_dict[phase2].extend(sols)

In [ ]:
def get_extended_line(x1, y1, x2, y2, ext=2, pts=100):
    m = (y2 - y1) / (x2 - x1)
    b = y1 - m * x1
    x_ext = np.linspace(min(x1, x2) - ext, max(x1, x2) + ext, pts)
    return x_ext, m * x_ext + b

# Create figure and axis
fig, ax = plt.subplots(figsize=(6,6))

# Plot polynomials
x_space = np.linspace(0, 1, 100)
fcc_a1 = [phase_poly_dict['FCC_A1'].subs({'x': x}) for x in x_space]
al5fe4 = [phase_poly_dict['AL5FE4'].subs({'x': x}) for x in x_space]
ax.plot(x_space, fcc_a1, 'b-', label='FCC_A1')
ax.plot(x_space, al5fe4, 'g-', label='AL5FE4')

# Process polynomial data using sympy's lambdify
x_sym = symbols('x')
fcc_a1_x = np.array(phase_values_dict['FCC_A1'])
fcc_a1_y = sp.lambdify(x_sym, phase_poly_dict['FCC_A1'], 'numpy')(fcc_a1_x)
al5fe4_x = np.array(phase_values_dict['AL5FE4'])
al5fe4_y = sp.lambdify(x_sym, phase_poly_dict['AL5FE4'], 'numpy')(al5fe4_x)

# Compute extended lines for the first and second points
x_line1, y_line1 = get_extended_line(fcc_a1_x[0], fcc_a1_y[0], al5fe4_x[0], al5fe4_y[0])
x_line2, y_line2 = get_extended_line(fcc_a1_x[1], fcc_a1_y[1], al5fe4_x[1], al5fe4_y[1])

# Plot points and extended lines
ax.plot(fcc_a1_x, fcc_a1_y, 'bo')
ax.plot(al5fe4_x, al5fe4_y, 'go')
ax.plot(x_line1, y_line1, 'r--')
ax.plot(x_line2, y_line2, 'm--')

# Format plot
ax.set(xlabel='X(FE)', ylabel='GM', xlim=(0,1), ylim=(-30000,5000), title='Al-Fe Polynomials')
ax.legend(loc='center left', bbox_to_anchor=(1, 0.6))
plt.show()

In [ ]:
# Create figure and axis
fig, ax = plt.subplots(figsize=(6,6))

# Plot phase data if available
for phase in ['FCC_A1', 'AL5FE4']:
    try:
        result = calculate(db, constituents, phase, T=298.15, P=101325, output='GM')
        ax.scatter(result.X.sel(component='FE'), result.GM, marker='.', s=20, 
                   color=color_dict[phase], label=phase)
    except KeyError:
        continue

# Process polynomial data using sympy's lambdify
x_sym = symbols('x')
fcc_a1_x = np.array(phase_values_dict['FCC_A1'])
fcc_a1_y = sp.lambdify(x_sym, phase_poly_dict['FCC_A1'], 'numpy')(fcc_a1_x)
al5fe4_x = np.array(phase_values_dict['AL5FE4'])
al5fe4_y = sp.lambdify(x_sym, phase_poly_dict['AL5FE4'], 'numpy')(al5fe4_x)

# Compute extended lines for the first and second points
x_line1, y_line1 = get_extended_line(fcc_a1_x[0], fcc_a1_y[0], al5fe4_x[0], al5fe4_y[0])
x_line2, y_line2 = get_extended_line(fcc_a1_x[1], fcc_a1_y[1], al5fe4_x[1], al5fe4_y[1])

# Plot points and extended lines
ax.scatter(fcc_a1_x, fcc_a1_y, color = color_dict['FCC_A1'])
ax.scatter(al5fe4_x, al5fe4_y, color = color_dict['AL5FE4'])
ax.plot(x_line1, y_line1, 'r--')
ax.plot(x_line2, y_line2, 'm--')

# Format plot
ax.set(xlabel='X(FE)', ylabel='GM', xlim=(0,1), ylim=(-30000,5000), title='Al-Fe Data')
ax.legend(loc='center left', bbox_to_anchor=(1, 0.6))
plt.show()

In [ ]:
phase_values_dict

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pycalphad import Database, variables as v, equilibrium
from pycalphad.plot.eqplot import eqplot

# Load the Al-Fe thermodynamic database (replace with a valid TDB file)
db = Database(r'../TDDatabaseFiles_temp/alfe.tdb')  # Ensure you have a valid TDB file

# Define components and phases
comps = ["AL", "FE", "VA"]  # Elements + Vacancy
phases = list(db.phases.keys())  # Use all available phases

# Remove the bad phases
for phase in ['AL13FE4', 'AL5FE2', 'B2_BCC', 'AL2FE']:
    if phase in phases:
        phases.remove(phase)

conditions = {v.T: 300, v.X("AL"): np.linspace(0, 1, 100), v.P: 101325}  # 1 atm

# Compute equilibrium
eq = equilibrium(db, comps, phases, conditions)

In [ ]:
eq

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pycalphad import Database, variables as v, equilibrium
from pycalphad.plot.eqplot import eqplot

# Load the Al-Fe thermodynamic database (replace with a valid TDB file)
db = Database(r'../TDDatabaseFiles_temp/alfe.tdb')  # Ensure you have a valid TDB file

# Define components and phases
comps = ["AL", "FE", "VA"]  # Elements + Vacancy
phases = list(db.phases.keys())  # Use all available phases

# Remove the bad phases
for phase in ['AL13FE4', 'AL5FE2', 'B2_BCC', 'AL2FE']:
    if phase in phases:
        phases.remove(phase)

conditions = {v.T: np.linspace(300, 2000, 100), v.X("AL"): np.linspace(0, 1, 100), v.P: 101325}  # 1 atm

# Compute equilibrium
eq = equilibrium(db, comps, phases, conditions)

In [ ]:
# Plot the phase diagram using eqplot (works for T-X diagrams)
fig = plt.figure(figsize=(8, 6))
eqplot(eq)
plt.title("Al-Fe Temperature-Composition Phase Diagram")
plt.show()